<a href="https://colab.research.google.com/github/okj3031402/LLM_study/blob/main/building_a_function_calling_chatbot_app_with_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28 # openai 라이브러리를 설치합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.0 MB/s eta 0:00:00


In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import json

# 함수 정의
def get_current_weather(date_time, location, unit="celsius"):
    """Get the current weather in a given location"""
    weather_info = {
        "date_time" : date_time,
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# 함수 설명 for LLM
functions = [
    {
        "name": "get_current_weather", # 함수 이름이 명확해야 합니다.
        "description": "Get the current weather in a given location", # 설명도 상세하게 작성해야 합니다.
        "parameters": {
            "type": "object",
            "properties": {
                "date_time" : {
                    "type": "string", # 타입
                    "description": "date and time", # 설명
                },
                "location": {
                    "type": "string", # 타입
                    "description": "The city and state, e.g. San Francisco, CA", # 설명
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["date_time", "location"], # 필수 항목
        },
    }
]

In [ ]:
# 함수 정의와 함수 설명을 매칭하기 위한 용도

available_functions = {
    "get_current_weather": get_current_weather,
}  # only one function in this example, but you can have multiple

In [ ]:
def llm(input_text, chat_history):

    if len(chat_history) == 0:
        chat_history.append({"role": "system", "content": "Act like a friend who is kind and highly empathetic. Respond to the user's input in a friendly and conversational manner in Korean"})

    chat_history.append({"role": "user", "content": input_text})

    print(chat_history)

    response = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613",
                                              messages = chat_history,
                                              functions = functions, # 편지봉투에 편지지 뿐만 아니라 함수 설명서도 동봉하자
                                              function_call = "auto",  # auto is default, but we'll be explicit
                                              )

    response_message = response["choices"][0]["message"]

    print("###")
    print(response_message)
    print("###")
    # function_call vs contents

    # GPT가 호출해야 할 함수를 특정했다면, 응답 편지봉투에 "function_call" 키 정보가 포함되어 응답이 작성됩니다.
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        # 함수 이름 가져오기
        function_name = response_message["function_call"]["name"]

        # 실제 호출 가능한 함수
        function_to_call = available_functions[function_name]

        # 함수 인자 가져오기
        function_args = json.loads(response_message["function_call"]["arguments"])

        # AI가 어떤 함수를 선택했고, 어떤 인자를 입력했는 지 chat_history에 추가합니다.
        chat_history.append(response_message)  # extend conversation with assistant's reply

        # 실제 함수를 호출해서 그 결과를 얻습니다.
        function_response = function_to_call(**function_args)

        # role function에 함수 호출 결과를 넘긴다.

        # Step 4: send the info on the function call and function response to GPT
        chat_history.append(
            {
                "role": "function", # 역할을 function으로 지정한다.
                "name": function_name,
                "content": function_response,
            }
        )

        # extend conversation with function response
        # 함수 호출 결과를 사용자 지시에 맞게 잘 설명해주는 용도로 호출
        second_response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo-0613",
            messages = chat_history,
        )

        output = second_response.choices[0].message.content
    else:
        output = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": output})

    return output

In [ ]:
def chat_with_user(user_message, chat_history):
    ai_message = llm(user_message, chat_history)
    return ai_message

chat_history = []

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message, chat_history)
    print(f" A I > {ai_message}")

USER > weather?
[{'role': 'system', 'content': "Act like a friend who is kind and highly empathetic. Respond to the user's input in a friendly and conversational manner in Korean"}, {'role': 'user', 'content': 'weather?'}]
###
{
  "role": "assistant",
  "content": "\uc548\ub155\ud558\uc138\uc694! \ud604\uc7ac \uc5b4\ub5a4 \ub3c4\uc2dc\uc758 \ub0a0\uc528\uc5d0 \uad00\uc2ec\uc774 \uc788\uc73c\uc2e0\uac00\uc694? \ub3c4\uc640\ub4dc\ub9b4\uac8c\uc694!"
}
###
 A I > 안녕하세요! 현재 어떤 도시의 날씨에 관심이 있으신가요? 도와드릴게요!
USER > 시애틀
[{'role': 'system', 'content': "Act like a friend who is kind and highly empathetic. Respond to the user's input in a friendly and conversational manner in Korean"}, {'role': 'user', 'content': 'weather?'}, {'role': 'assistant', 'content': '안녕하세요! 현재 어떤 도시의 날씨에 관심이 있으신가요? 도와드릴게요!'}, {'role': 'user', 'content': '시애틀'}]
###
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"date_time\": \"now\",\n  \"location